# Mask project
This project is trying to reach the goal of detecting if a person is using or not the mask

## Initialize parameters

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

epoch = 100 # number of iterations
width = 160 # width of the image in pixels
height = 160 # height of the image in pixels
categories = 2 # 1 (mask) or 0 (without mask)
test_size = 0.3 # percentage of the data that is going to be for testing


## Loading the data
We load the data from the dataset

In [ ]:
# Create the data lists

final_images = []
labels = []

# get the current directory and i move until our dataset
data_folder = os.path.join(os.getcwd(), "drive", "MyDrive", "Colab Notebooks", "TF", "Module_2", "Labs", "2", "dataset")
# list the folders (0) => without mask / (1) => with mask
folders = os.listdir(data_folder)

# Iterate over both folders
for folder in folders:
  print("folder ", folder)
  # each folder
  dir_folder = os.path.join(data_folder,folder)
  # List the images
  images =  os.listdir(dir_folder)
  # iterate over all the images in the folder
  for image in images:
    # Read the image
    img_dir = cv2.imread(os.path.join(dir_folder, image))
    # grayscale
    img_gray = cv2.cvtColor(img_dir, cv2.COLOR_BGR2GRAY)
    # equalizehist
    img_gray = cv2.equalizeHist(img_gray)
    # resize the image
    img_res = cv2.resize(img_gray, (width, height))
    # Save the image in the matrix
    final_images.append(img_res)
    # Save the labels of the images
    if folder == "0":
      #print("0")
      labels.append([1,0])
    else:
      #print("1")
      labels.append([0,1])

folder  1
folder  0


## Dividing the data


In [ ]:
#labels =  tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(final_images), np.array(labels), test_size=test_size
)
# reshape the training data
x_train =  np.reshape(x_train, (-1, width, height, 1))
# reshape the test data 
x_test =  np.reshape(x_test, (-1, width, height, 1))
# Normalizar los valores 
x_test = x_test/255
# Normalizar los valores
x_train = x_train/255



## Create the model

In [ ]:

# initialize the model
model = tf.keras.models.Sequential()

# First convolution -> 32 filters with a kernel of 2x2 and an activation of type "relu"
# First MaxPooling
model.add(tf.keras.layers.Conv2D(32, (2,2), activation="relu", input_shape=(width, height, 1), padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# Second convolution -> 64 filters with a kernel of 3x3 and an activation of type "relu"
# Second MaxPooling
model.add(tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))

# Third convolution -> 128 filters with a kernel of 3x3 and an activation of type "relu"
# Third MaxPooling
model.add(tf.keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))

# Fourth convolution -> 256 filters with a kernel of 3x3 and an activation of type "relu"
# Fourth MaxPooling
model.add(tf.keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))

# Fifth convolution -> 60 filters with a kernel of 2x2 and an activation of type "relu"
# Fifth MaxPooling
model.add(tf.keras.layers.Conv2D(512, (3,3), activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))


# Dropout layer (prevent overfitting) -> 20%
model.add(tf.keras.layers.Dropout(0.2))

# Flatten layer
model.add(tf.keras.layers.Flatten())

# First Fully connected layer -> 500 neurons
model.add(tf.keras.layers.Dense(500, activation="relu"))

# Dropout layer (prevent overfitting) -> 20%
model.add(tf.keras.layers.Dropout(0.2))

# Second Fully connected layer -> 500 neurons
model.add(tf.keras.layers.Dense(500, activation="relu"))

# Dropout layer (prevent overfitting) -> 20%
model.add(tf.keras.layers.Dropout(0.2))

# Output layer -> 2 neurons (with mask - without mask) / activation Softmax
model.add(tf.keras.layers.Dense(categories,activation="softmax"))

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 160, 32)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0

## Train the model

In [ ]:
model.fit(x_train, y_train, epochs=epoch)

## Evaluate the model

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

62/62 - 18s - loss: 0.2502 - accuracy: 0.9611


[0.25021225214004517, 0.9610521197319031]

## Save the model

In [ ]:
model.save("modelo2.hdf5")